In [15]:
import pickle
import numpy as np
from google.colab import files
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [16]:
# Fazer o upload do arquivo treinado
uploaded = files.upload()

Saving tokenizer.pkl to tokenizer.pkl
Saving nome_do_grupo_RNN.keras to nome_do_grupo_RNN (2).keras


In [17]:
# Carregar o modelo treinado
model = load_model("nome_do_grupo_RNN.keras")

In [18]:
# Carregar o tokenizer
with open('nome_do_grupo_tokenizer_RNN.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

total_palavras = len(tokenizer.word_index) + 1
max_seq_len = model.input_shape[1]

In [19]:
# Função de autocompletar
def autocompletar_texto(seed_text, n_palavras):
    for _ in range(n_palavras):
        tokens = tokenizer.texts_to_sequences([seed_text])[0]
        tokens_padded = pad_sequences([tokens], maxlen=max_seq_len, padding='pre')
        predicted = np.argmax(model.predict(tokens_padded, verbose=0), axis=-1)

        output_word = ''
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        if output_word == '':
            break
        seed_text += ' ' + output_word
    return seed_text

In [28]:
# Frases para avaliação
frases_para_avaliar = [
    "eu gosto",
    "ela gosta",
    "ele gosta",
    "nós gostamos",
    "vocês gostam",
    "o gato está em"
]

In [33]:
# Gabarito esperado
continuacoes_corretas = [
    "de programar",
    "de correr",
    "de jogar futebol",
    "de estudar",
    "de viajar",
    "cima do muro"
]

In [34]:
# Avaliação: autocompletando 3 palavras
# Função de avaliação com cálculo de acurácia
total_palavras = 0
total_acertos = 0

for frase, gabarito in zip(frases_para_avaliar, continuacoes_corretas):
    completada = autocompletar_texto(frase, n_palavras=3)
    print(f'Entrada: "{frase}" -> Autocompletada: "{completada}"')

    # Extraindo apenas a parte completada
    completada_palavras = completada.split()[len(frase.split()):]  # pega só as palavras que foram completadas
    gabarito_palavras = gabarito.split()

    # Comparando
    acertos = sum([pred == real for pred, real in zip(completada_palavras, gabarito_palavras)])

    total_palavras += len(gabarito_palavras)
    total_acertos += acertos

    # Acurácia por frase
    acuracia_frase = acertos / len(gabarito_palavras)
    print(f'Gabarito: "{gabarito}" -> Acurácia: {acuracia_frase:.2f}\n')

# Acurácia total
acuracia_total = (total_acertos / total_palavras)*100
print(f'\nAcurácia total: {acuracia_total:.2f}%')

Entrada: "eu gosto" -> Autocompletada: "eu gosto gato gato está"
Gabarito: "de programar" -> Acurácia: 0.00

Entrada: "ela gosta" -> Autocompletada: "ela gosta gato gato está"
Gabarito: "de correr" -> Acurácia: 0.00

Entrada: "ele gosta" -> Autocompletada: "ele gosta gato gato está"
Gabarito: "de jogar futebol" -> Acurácia: 0.00

Entrada: "nós gostamos" -> Autocompletada: "nós gostamos gato gato está"
Gabarito: "de estudar" -> Acurácia: 0.00

Entrada: "vocês gostam" -> Autocompletada: "vocês gostam gato gato está"
Gabarito: "de viajar" -> Acurácia: 0.00

Entrada: "o gato está em" -> Autocompletada: "o gato está em cima do muro"
Gabarito: "cima do muro" -> Acurácia: 1.00


Acurácia total: 21.43%
